### Example code to show how to use the exsi client independently from a jupyter notebook

In [1]:
import os
import sys
import json

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.path.abspath(''), '..')))

from shimTool.exsiProcess import *

def load_config(filename):
    with open(filename, 'r') as file:
        return json.load(file)

#### first load the exsi client and wait for it to connect

In [3]:
config = load_config('../config.json')

# Make a sample EXSI class instance
# The requireExsiConnection decorator will check if the connection is ready before executing any exsi functionality.
exsi_instance = exsi(config)
exsi_instance.connected_ready_event.wait() # wait for the scanner to be connected

INFO EXSI CLIENT: Socket connected
EXSI CLIENT DEBUG: Processing command:  ConnectToScanner product=newHV passwd=rTpAtD
EXSI CLIENT DEBUG: Processing command:  NotifyEvent all=on
EXSI CLIENT DEBUG: Processing command:  GetExamInfo


True

#### now you can queue commands to it and have it execute actions on the scanner as soon as the scanner is available.
##### do note that if you want to access variables from the exsiClient that get set after some command executes, you will have to use the relevant condition variables to sync up with when that event actually occurs. below is an example of getting the task keys that are set by a specific protocol "BPT_EXSI"

In [4]:
# Load protocol and get task keys
protocol_name = "BPT_EXSI"
exsi_instance.sendLoadProtocol(protocol_name)
exsi_instance.send_event.wait() # wait for a command to be sent to scanner
exsi_instance.ready_event.wait()# wait for scanner to return success response of the last command
# use exsi_instance.sendWaitForImagesCollected() to wait for images to be collected on the scanner
print(f"tasks: {exsi_instance.taskKeys}")

print('Localizer scan loaded')
# Run localizer
exsi_instance.sendSelTask() # this queues the next task in line, it pops the task off a queue that is created when protocol is loaded
exsi_instance.sendSelTask()

#... add commands as you like

EXSI CLIENT DEBUG: Loading protocol BPT_EXSI
tasks: [103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113]
Localizer scan loaded


EXSI CLIENT DEBUG: Processing command:  LoadProtocol site path="BPT_EXSI"
EXSI CLIENT DEBUG: Task keys found in message:  [103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113]
EXSI CLIENT DEBUG: Processing command:  SelectTask taskkey=103
EXSI CLIENT DEBUG: Processing command:  SelectTask taskkey=104


## This is for multi processing, but it like doesn't really work so i dunno maybe should remove this.

In [ ]:
process, request_queue, response_queue, parent_pipe, stop_event = start_exsi_client()

stdout_thread = threading.Thread(target=read_stdout, args=(parent_pipe,))
stdout_thread.daemon = True
stdout_thread.start()

# result = call_method(request_queue, response_queue, 'sendGetExamInfo')
# print("Result:", result)

result = call_method(request_queue, response_queue, 'sendLoadProtocol', 'BPT_EXSI')
print("Result:", result)

In [ ]:
result = call_method(request_queue, response_queue, 'sendLoadProtocol', 'BPT_EXSI')
print("Result:", result)

In [ ]:


stop_event.set()
process.join()
parent_pipe.close()